In [ ]:
import requests
from datetime import datetime, timedelta, timezone

# === Configuration ===
API_KEY = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6IjdiYjVkMjVlMjhmMDQ0ZmQ5ZGZiYmFiMDI5YmIzMGU3IiwiaCI6Im11cm11cjEyOCJ9"  # Replace with your real KNMI API key
BASE_URL = "https://api.dataplatform.knmi.nl/edr/v1"
COLLECTION = "10-minute-in-situ-meteorological-observations"
station_id = "0-20000-0-06203"

# === Time window: last 30 minutes ===
def valid_datetime_input(timepoint: datetime):
    return timepoint.isoformat(timespec="seconds").replace("+00:00", "Z")


now = datetime.now(timezone.utc)
start = now - timedelta(minutes=30)
time_range = f"{valid_datetime_input(start)}/{valid_datetime_input(now)}"
now = valid_datetime_input(now)

print(time_range)
# === API request ===
params = {
    "datetime": f"P1D/{time_range}",
    "parameter-name": "air_pressure_at_sea_level",
    "crs": "EPSG:4326"
}

headers = {
    "Authorization": API_KEY
}


# List available collections to verify the collection name
response = requests.get(
    f"{BASE_URL}/collections/{COLLECTION}/locations/{station_id}",
    headers=headers,
    params=params,
)
response.raise_for_status()
data = response.json()

data


In [ ]:
features = data.get("features", [])
if not features:
    print("No data found.")
else:
    print(f"Found {len(features)} measurements for air pressure.\n")
    for f in features:
        station = f.get("id")
        coords = f.get("geometry", {}).get("coordinates", [])
        props = f.get("properties", {})
        time = props.get("datetime")

        pressure_obj = props.get("parameter", {}).get("air_pressure_at_sea_level")
        pressure = pressure_obj.get("value") if pressure_obj else None

        print(f"Station: {station}")
        print(f"  Time: {time}")
        print(f"  Pressure: {pressure} hPa")
        print(f"  Location (lon, lat): {coords[:2]}")
        print("-" * 30)


In [ ]:
import json
print(json.dumps(data["features"][0], indent=2))